# What is a "match"? (Part 1)

---
*note*

*Other "parts" will be linked here if they are made.*

*This follows my thought process on trying to figure out how best to "match"*
*candidates to labels. It might be a bit scattered and change over time.*

---

This notebook explores whether or not two strings are considered as a "match".
This is important to how we calculate the statistics for model evaluation and
comparison. Importantly, for every set of candidate labels (produced by the
model) and set of document labels (manually produced) we need to calculate: true
positives (TP), false positives (FP), and false negatives (FN) (true negatives
are not important here). 

Let's consider the following example, where the "Target List" labels are taken
from the `7d9c76c33.json` document in the private dataset. The "Candidate List"
is not from a model and is only there to highlight this example.

| Candidate List (CL)                     | Target List (TL)                                |
| --------------------------------------- | ----------------------------------------------- |
| 1. Database of Genotypes and Phenotypes | 1. Database of Genotypes and Phenotypes         |
| 2. DGP                                  | 2. dbGap                                        |
| 3. 1,000 Genomes Project                | 3. database of Genotypes and Phenotypes (dbGap) |
|                                         | 4. 1,000 Genomes projects                       |
|                                         | 5. 1,000 Genomes reference data                 |

In evaluating a model True Positives, False Positives, and False Negatives can
be assigned in the following ways:

1. True Positives are only assigned to entities in the Target List
2. False Positives are only assigned to entities in the Candidate List
3. False Negatives are only assinged to entities in the Target List

The initial scope of this project is at the document-level. Additionally the aim
is to assess whether or not a method sucessfully extracts dataset mentions from
a text. By eye, there seems to be an intuitive agreement between the Candidate
and Target lists. However, a challenge arises when we need to assign True
Positives, False Positives, and False Negatives.

First, for the Database of Genotypes and Phenotypes we have at least one single
clear match, CL1 → TL1. However, entry CL1 is also a match to TL2 and TL3, even
if not exact. Further, CL2 seems to be an incorrect, but reasonable acronym, for 
TL[1,2,3]. Finally, CL3 is an inexact match to TL[4,5]

For our purposes, we need to develop a matching scheme that will flexibly assign
assign TP labels to alls of the entities in the TL and FP to CL2

In [1]:
candidate_list = [
    "Database of Genotypes and Phenotypes",
    "DGP",
    "1,000 Genomes Project",
]

target_list = [
    "Database of Genotypes and Phenotypes",
    "dbGap",
    "database of Genotypes and Phenotypes (dbGap)",
    "1,000 Genomes projects",
    "1,000 Genomes reference data",
]

## Resolving True Positives

True Positives are those entries in the Target List that can be matched to
atleast one entry in the Candidate List. We only want to count Target List
entries to avoid counting multiple references to the same dataset at the
document level. 

### `thefuzz`

First let's try the popular fuzzy matching library `fuzzywuzzy` now called 
`thefuzz`.

It uses [Levenshtein Distance](https://en.wikipedia.org/wiki/Levenshtein_distance) to compare two strings, which is defined below (source wikipedia):




`thefuzz` uses a `ratio` as a score for how similar two strings are. The ratio as calculated in [levenshtein-py](https://github.com/ztane/python-Levenshtein/blob/master/Levenshtein/_levenshtein.c#L780) is:

$$\textrm{ratio}(a, b) = \frac{(\max(|a|, |b|) - \textrm{lev}(a,b))}{(\max(|a|, |b|))}$$

The ratio describes 1 - "levenshtein distance normalized by string length". This is then multiplied by 100 to get a score.

In [2]:
from thefuzz import fuzz

In [3]:
for target in target_list:
    output_str = "ratio('{}', '{}') = {}"
    for candidate in candidate_list:
        print(output_str.format(target, candidate, fuzz.ratio(target, candidate)))

ratio('Database of Genotypes and Phenotypes', 'Database of Genotypes and Phenotypes') = 100
ratio('Database of Genotypes and Phenotypes', 'DGP') = 15
ratio('Database of Genotypes and Phenotypes', '1,000 Genomes Project') = 39
ratio('dbGap', 'Database of Genotypes and Phenotypes') = 20
ratio('dbGap', 'DGP') = 25
ratio('dbGap', '1,000 Genomes Project') = 8
ratio('database of Genotypes and Phenotypes (dbGap)', 'Database of Genotypes and Phenotypes') = 88
ratio('database of Genotypes and Phenotypes (dbGap)', 'DGP') = 9
ratio('database of Genotypes and Phenotypes (dbGap)', '1,000 Genomes Project') = 34
ratio('1,000 Genomes projects', 'Database of Genotypes and Phenotypes') = 38
ratio('1,000 Genomes projects', 'DGP') = 8
ratio('1,000 Genomes projects', '1,000 Genomes Project') = 93
ratio('1,000 Genomes reference data', 'Database of Genotypes and Phenotypes') = 34
ratio('1,000 Genomes reference data', 'DGP') = 6
ratio('1,000 Genomes reference data', '1,000 Genomes Project') = 73


Fuzzy matching using the Levenshtein distance seems to work well for:

`ratio('Database of Genotypes and Phenotypes', 'Database of Genotypes and Phenotypes') = 100`

but is somewhat lacking when naively applied to:

`ratio('1,000 Genomes projects', '1,000 Genomes Project') = 93`

and 

`ratio('database of Genotypes and Phenotypes (dbGap)', 'Database of Genotypes and Phenotypes') = 88`

These likely should be a little bit higer as we probably don't care about casing
and the additional parenthesis.

with some preprocessing we could get a little closer. `thefuzz` has a utility
for this called `process` which allows us to write our own `processor` that can
apply some transformations to a string before its gets matched. We can also
write a custom `scorer` which should have a signature like:

$$f(a, b) \mapsto x : 0 \leq x \leq 100$$

For now let's just try making everything lower case and keep the same scorer.


In [4]:
from functools import partial
from thefuzz import process


def processor_lowercase(s):
    return s.lower()


for target in target_list:
    output_str = "f('{}', '{}') = {}"
    for candidate, score in process.extract(
        target, candidate_list, processor=processor_lowercase, scorer=fuzz.ratio
    ):
        print(output_str.format(target, candidate, score))

f('Database of Genotypes and Phenotypes', 'Database of Genotypes and Phenotypes') = 100
f('Database of Genotypes and Phenotypes', '1,000 Genomes Project') = 39
f('Database of Genotypes and Phenotypes', 'DGP') = 15
f('dbGap', 'DGP') = 75
f('dbGap', 'Database of Genotypes and Phenotypes') = 24
f('dbGap', '1,000 Genomes Project') = 15
f('database of Genotypes and Phenotypes (dbGap)', 'Database of Genotypes and Phenotypes') = 90
f('database of Genotypes and Phenotypes (dbGap)', '1,000 Genomes Project') = 34
f('database of Genotypes and Phenotypes (dbGap)', 'DGP') = 13
f('1,000 Genomes projects', '1,000 Genomes Project') = 98
f('1,000 Genomes projects', 'Database of Genotypes and Phenotypes') = 41
f('1,000 Genomes projects', 'DGP') = 16
f('1,000 Genomes reference data', '1,000 Genomes Project') = 73
f('1,000 Genomes reference data', 'Database of Genotypes and Phenotypes') = 34
f('1,000 Genomes reference data', 'DGP') = 6


We got an improvement in our scores: 

`f('1,000 Genomes projects', '1,000 Genomes Project') = 93 → 98`

and

`f('database of Genotypes and Phenotypes (dbGap)', 'Database of Genotypes and Phenotypes') = 88 → 90`


However, the following change also happened:

`ratio('dbGap', 'DGP') = 25 → 75` 

which may not serve us well in the long run.

Let's try again, but this time using a different scorer: `partial_ratio`. This
scorer is a little more forgiving and will match substrings. The substrings are
extracted using
[difflib.SequenceMatcher.get_matching_blocks](https://docs.python.org/3/library/difflib.html#difflib.SequenceMatcher.get_matching_blocks).
`get_matching_blocks` will extract matching string subsets, which
`partial_ratio` uses to find the best subset of the longer of the two strings to
compare with the shorter of the two strings. For example (taken from
[code/docs](https://github.com/seatgeek/thefuzz/blob/6e68af84e086b3e5f7253d4f9b0d6c7313e34637/thefuzz/fuzz.py#L49)):

```python
s1 = "abcd"
s2 = "XXXbcdeEEE"
``` 
`partial_ratio` will compare `"abcd"` against `"Xbcd"`

In [5]:
from functools import partial
from thefuzz import process


def processor_lowercase(s):
    return s.lower()


for target in target_list:
    output_str = "f('{}', '{}') = {}"
    for candidate, score in process.extract(
        target, candidate_list, processor=processor_lowercase, scorer=fuzz.partial_ratio
    ):
        print(output_str.format(target, candidate, score))

f('Database of Genotypes and Phenotypes', 'Database of Genotypes and Phenotypes') = 100
f('Database of Genotypes and Phenotypes', '1,000 Genomes Project') = 48
f('Database of Genotypes and Phenotypes', 'DGP') = 33
f('dbGap', 'DGP') = 67
f('dbGap', 'Database of Genotypes and Phenotypes') = 40
f('dbGap', '1,000 Genomes Project') = 20
f('database of Genotypes and Phenotypes (dbGap)', 'Database of Genotypes and Phenotypes') = 100
f('database of Genotypes and Phenotypes (dbGap)', '1,000 Genomes Project') = 48
f('database of Genotypes and Phenotypes (dbGap)', 'DGP') = 33
f('1,000 Genomes projects', '1,000 Genomes Project') = 100
f('1,000 Genomes projects', 'Database of Genotypes and Phenotypes') = 45
f('1,000 Genomes projects', 'DGP') = 33
f('1,000 Genomes reference data', '1,000 Genomes Project') = 76
f('1,000 Genomes reference data', 'Database of Genotypes and Phenotypes') = 40
f('1,000 Genomes reference data', 'DGP') = 33


This markedly improved our scores:

`f('1,000 Genomes projects', '1,000 Genomes Project') = 98 → 100`

and

`f('database of Genotypes and Phenotypes (dbGap)', 'Database of Genotypes and Phenotypes') = 90 → 100`


And delightfully, the following change also happened:

`ratio('dbGap', 'DGP') = 75 → 67` 


So if we return to our table. We can now mark the following as <span style="color:green">true positives</span>:

| Candidate List (CL)                     | Target List (TL)                                                                 |
| --------------------------------------- | -------------------------------------------------------------------------------- |
| 1. Database of Genotypes and Phenotypes | 1. <span style="color:green">Database of Genotypes and Phenotypes</span>         |
| 2. DGP                                  | 2. dbGap                                                                         |
| 3. 1,000 Genomes Project                | 3. <span style="color:green">database of Genotypes and Phenotypes (dbGap)</span> |
|                                         | 4. <span style="color:green">1,000 Genomes projects</span>                       |
|                                         | 5. 1,000 Genomes reference data                                                  |


The main issue that we have now is that we're not matching TL[2,5] even though
they seemed to be pretty close. The unconventional style of the TL2 may just
be a casuality of the method, but it's not a completely uncommon style. TL5
is a bit trickier.


In [6]:
true_positives = [
    "Database of Genotypes and Phenotypes",
    "database of Genotypes and Phenotypes (dbGap)",
    "1,000 Genomes projects",
]

## Resolving False Negatives

False Negatives are those entries in the Target List that can't be matched to
atleast one entry in the Candidate List. This is much simpler to resolve though
it is dependent of the efficacy of the methodology for defining True Positives.

So if we return to our table. We can now mark the following as <span style="color:red">false negatives</span>:

| Candidate List (CL)                     | Target List (TL)                                                                 |
| --------------------------------------- | -------------------------------------------------------------------------------- |
| 1. Database of Genotypes and Phenotypes | 1. <span style="color:green">Database of Genotypes and Phenotypes</span>         |
| 2. DGP                                  | 2. <span style="color:red">dbGap</span>                                          |
| 3. 1,000 Genomes Project                | 3. <span style="color:green">database of Genotypes and Phenotypes (dbGap)</span> |
|                                         | 4. <span style="color:green">1,000 Genomes projects</span>                       |
|                                         | 5. <span style="color:red">1,000 Genomes reference data</span>                   |

In [7]:
false_negatives = list(set(target_list) - set(true_positives))

## Resolving False Positives

False Positives are those entries in the Candidate List that can't be matched to
atleast one entry in the Target List. 

We can do this more performantly by keeping track when we do the True Positive
calculation, but for simplicity, we'll do it again here.

In [8]:
def processor_lowercase(s):
    return s.lower()


false_positives = list(candidate_list)
for target in target_list:
    output_str = "f('{}', '{}') = {}"
    for candidate, score in process.extract(
        target, candidate_list, processor=processor_lowercase, scorer=fuzz.partial_ratio
    ):
        if score > 90 and candidate in false_positives:
            false_positives.remove(candidate)

print(false_positives)

['DGP']



So if we return to our table. We can now mark the following as <span style="color:purple">false positives</span>:

| Candidate List (CL)                     | Target List (TL)                                                                 |
| --------------------------------------- | -------------------------------------------------------------------------------- |
| 1. Database of Genotypes and Phenotypes | 1. <span style="color:green">Database of Genotypes and Phenotypes</span>         |
| 2. <span style="color:purple">DGP</span>| 2. <span style="color:red">dbGap</span>                                          |
| 3. 1,000 Genomes Project                | 3. <span style="color:green">database of Genotypes and Phenotypes (dbGap)</span> |
|                                         | 4. <span style="color:green">1,000 Genomes projects</span>                       |
|                                         | 5. <span style="color:red">1,000 Genomes reference data</span>                   |

### Further Thoughts

There might be a better way to do this, but I think it just needs to be clear
what is happening and what things can be changed about how things are matched.

Additionally, when working at the document-level, it is probably worthwhile to
consider the problem of resolving mutiple references to the same dataset in the
same document, but where one reference would be recognized as a true positive
and another would be recognized as a false negative.

### A Single Function That Caclulates TP, FP, FN

In [9]:
from typing import Callable, List, Optional, Tuple
from thefuzz import process


def retrieve_tpfpfn(
    candidate_list: List[str],
    target_list: List[str],
    scorer: Optional[Callable[[str, str], int]] = None,
    processor: Optional[Callable[[str], str]] = None,
    min_score: int = 90,
    top_n: int = 5,
) -> Tuple[List[str], List[str], List[str]]:
    """Calculate true positives, false positives, and false negatives."""

    true_positives = []
    false_negatives = list(target_list)
    false_positives = list(candidate_list)
    for target in target_list:
        extracted = process.extract(
            target, candidate_list, processor=processor, scorer=scorer, limit=top_n
        )
        for candidate, score in filter(lambda x: x[1] > min_score, extracted):
            if candidate in false_positives:
                false_positives.remove(candidate)
            if target not in true_positives:
                true_positives.append(target)
            if target in false_negatives:
                false_negatives.remove(target)

    return true_positives, false_positives, false_negatives

Test the function

In [10]:
candidate_list = [
    "Database of Genotypes and Phenotypes",
    "DGP",
    "1,000 Genomes Project",
]

target_list = [
    "Database of Genotypes and Phenotypes",
    "dbGap",
    "database of Genotypes and Phenotypes (dbGap)",
    "1,000 Genomes projects",
    "1,000 Genomes reference data",
]


def processor_lowercase(s):
    return s.lower()


true_positives, false_positives, false_negatives = retrieve_tpfpfn(
    candidate_list, target_list, fuzz.partial_ratio, processor_lowercase
)
print("true_positives: {}".format(true_positives))
print("false_positives: {}".format(false_positives))
print("false_negatives: {}".format(false_negatives))

true_positives: ['Database of Genotypes and Phenotypes', 'database of Genotypes and Phenotypes (dbGap)', '1,000 Genomes projects']
false_positives: ['DGP']
false_negatives: ['dbGap', '1,000 Genomes reference data']


Further reading:

[Fuzzy Matching 101 (datalatter.com)](https://dataladder.com/fuzzy-matching-101/)